# 🩺 GenECG Inference Demo

**Feed an ECG image → Get SNOMED-CT diagnostic codes**

This notebook uses your trained ViT model to predict diagnoses.

---

In [ ]:
# Setup
!pip install -q transformers torch torchvision pillow

from google.colab import drive
drive.mount('/content/drive')

import os
PROJECT_ROOT = "/content/drive/Othercomputers/My_MacBook_Air/GenECG-Diagnostic"
os.chdir(PROJECT_ROOT)
print(f"✅ Working directory: {os.getcwd()}")

In [ ]:
import torch
from transformers import ViTForImageClassification
from PIL import Image
from torchvision import transforms
from pathlib import Path

# Load trained model
print("📦 Loading ViT model...")
checkpoint = torch.load("runs/vit/vit_multilabel_checkpoint.pt", map_location='cpu')

model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=checkpoint['num_classes'],
    ignore_mismatched_sizes=True,
)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

snomed_cols = checkpoint['snomed_cols']
print(f"✅ Model loaded with {len(snomed_cols)} SNOMED classes")

In [ ]:
def predict_ecg(image_path, top_k=10, threshold=0.3):
    """Predict SNOMED-CT codes for an ECG image."""
    
    # Transform
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    # Load image
    image = Image.open(image_path).convert('RGB')
    img_tensor = transform(image).unsqueeze(0)
    
    # Predict
    with torch.no_grad():
        logits = model(img_tensor).logits
        probs = torch.sigmoid(logits)[0].numpy()
    
    # Get top predictions
    top_indices = probs.argsort()[::-1][:top_k]
    
    print("="*60)
    print("📋 SNOMED-CT DIAGNOSTIC PREDICTIONS")
    print("="*60)
    print(f"{'SNOMED Code':<15} {'Probability':>12} {'Status':<12}")
    print("-"*60)
    
    positive_count = 0
    for idx in top_indices:
        code = snomed_cols[idx].replace('SNOMED_', '')
        prob = probs[idx]
        status = "🔴 POSITIVE" if prob >= threshold else ""
        if prob >= threshold:
            positive_count += 1
        print(f"{code:<15} {prob:>11.1%} {status:<12}")
    
    print("-"*60)
    print(f"\n📊 Summary: {positive_count} diagnoses above {threshold:.0%} threshold")
    
    return probs

In [ ]:
# 🔍 TEST ON A SAMPLE IMAGE
# Change this path to test different images!

test_image = "data/Raw/GenECG/Dataset_A_ECGs_without_imperfections/00000/00001_hr_1R.png"

# Show the image
from IPython.display import display
img = Image.open(test_image)
display(img.resize((600, 400)))

# Predict!
probs = predict_ecg(test_image, top_k=10, threshold=0.3)

In [ ]:
# 🔍 TEST ON ANOTHER IMAGE
test_image2 = "data/Raw/GenECG/Dataset_A_ECGs_without_imperfections/00000/00050_hr_1R.png"

img = Image.open(test_image2)
display(img.resize((600, 400)))

probs = predict_ecg(test_image2, top_k=10, threshold=0.3)

In [ ]:
# 📤 UPLOAD YOUR OWN ECG IMAGE
from google.colab import files

print("📤 Upload an ECG image to analyze:")
uploaded = files.upload()

for filename in uploaded.keys():
    print(f"\n🔍 Analyzing: {filename}")
    img = Image.open(filename)
    display(img.resize((600, 400)))
    probs = predict_ecg(filename, top_k=10, threshold=0.3)

---
## 📋 SNOMED-CT Code Reference

The model predicts from 65 diagnostic codes. Example codes:
- **4065279** - Normal ECG
- **4247796** - Sinus rhythm
- **4329847** - Atrial fibrillation
- etc.

Look up full descriptions at: https://browser.ihtsdotools.org/